In [1]:
import torch
import numpy as np
from tqdm import tqdm
import time
import os
import sys
import pprint
import pickle  # For saving and loading intermediate results

In [2]:
pprint.pprint(sys.path)

['C:\\Users\\hirom\\NLP\\LLM\\fcc-gpt-course',
 'C:\\Users\\hirom\\anaconda3\\python39.zip',
 'C:\\Users\\hirom\\anaconda3\\DLLs',
 'C:\\Users\\hirom\\anaconda3\\lib',
 'C:\\Users\\hirom\\anaconda3',
 '',
 'C:\\Users\\hirom\\AppData\\Roaming\\Python\\Python39\\site-packages',
 'C:\\Users\\hirom\\anaconda3\\lib\\site-packages',
 'C:\\Users\\hirom\\anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\hirom\\anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\hirom\\anaconda3\\lib\\site-packages\\Pythonwin']


In [3]:
sys.path.append('D:\\NLP\\JParaCrawl')
sys.path.append('C:\\users\\hirom\\nlp\\llm\cuda\\lib\\site-packages')

In [4]:
pprint.pprint(sys.path)

['C:\\Users\\hirom\\NLP\\LLM\\fcc-gpt-course',
 'C:\\Users\\hirom\\anaconda3\\python39.zip',
 'C:\\Users\\hirom\\anaconda3\\DLLs',
 'C:\\Users\\hirom\\anaconda3\\lib',
 'C:\\Users\\hirom\\anaconda3',
 '',
 'C:\\Users\\hirom\\AppData\\Roaming\\Python\\Python39\\site-packages',
 'C:\\Users\\hirom\\anaconda3\\lib\\site-packages',
 'C:\\Users\\hirom\\anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\hirom\\anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\hirom\\anaconda3\\lib\\site-packages\\Pythonwin',
 'D:\\NLP\\JParaCrawl',
 'C:\\users\\hirom\\nlp\\llm\\cuda\\lib\\site-packages']


In [8]:
from transformers import BertTokenizer, BertModel

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

In [10]:
def load_data(file_path):
    lines = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            lines.append(line.strip())
    return lines

en_train_path = 'D:\\NLP\\JParaCrawl\\JP_en_train.txt'
en_test_path = 'D:\\NLP\\JParaCrawl\\JP_en_test.txt'
ja_train_path = 'D:\\NLP\\JParaCrawl\\JP_ja_train.txt'
ja_test_path = 'D:\\NLP\\JParaCrawl\\JP_ja_test.txt'

en_train = load_data(en_train_path)
en_test = load_data(en_test_path)
ja_train = load_data(ja_train_path)
ja_test = load_data(ja_test_path)

In [11]:
start_time = time.time()

def encoding(text_list, tokenizer):
    encoded_texts = []
    for text in tqdm(text_list, desc="Processing texts", leave=True):
        encoded_text = tokenizer(text, add_special_tokens=True, padding=False, truncation=False, return_tensors='pt')
        encoded_texts.append(encoded_text)
    return encoded_texts


def process_and_save_chunks(data, tokenizer, num_chunks=20, prefix='chunk', save_dir='.'):
    """Process data in chunks, save each chunk, and clear from memory."""
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    chunk_size = len(data) // num_chunks
    for i in range(num_chunks):
        start_idx = i * chunk_size
        end_idx = (i + 1) * chunk_size if i != num_chunks - 1 else len(data)

        chunk = data[start_idx:end_idx]
        encoded_chunk = encoding(chunk, tokenizer)

        # Save chunk to file
        file_path = os.path.join(save_dir, f'{prefix}_encoded_chunk_{i}.pkl')
        with open(file_path, 'wb') as f:
            pickle.dump(encoded_chunk, f)

        # Clear memory
        del chunk
        del encoded_chunk
        torch.cuda.empty_cache()  # If using GPU


save_dir = 'D:\\NLP\\JParaCrawl\\encoded_data'


process_and_save_chunks(ja_train, tokenizer, num_chunks=20, prefix='ja_train', save_dir=save_dir)
process_and_save_chunks(ja_test, tokenizer, num_chunks=20, prefix='ja_test', save_dir=save_dir)
process_and_save_chunks(en_train, tokenizer, num_chunks=20, prefix='en_train', save_dir=save_dir)
process_and_save_chunks(en_test, tokenizer, num_chunks=20, prefix='en_test', save_dir=save_dir)


        

"""
encoded_ja_train = encoding(ja_train, tokenizer)
encoded_ja_test = encoding(ja_test, tokenizer)
encoded_en_train = encoding(en_train, tokenizer)
encoded_en_test = encoding(en_test, tokenizer)
"""


end_time = time.time()
processing_time = end_time - start_time

def format_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = seconds % 60
    return f"Processing time: {hours}h {minutes}m {seconds:.2f}s"

print(format_time(processing_time))


Processing texts: 100%|████████████████████████████████████████████████████████| 43820/43820 [00:27<00:00, 1592.51it/s]


Processing texts: 100%|████████████████████████████████████████████████████████| 43820/43820 [00:27<00:00, 1598.30it/s]


Processing time: 4h 33m 31.01s
